# Turmell-Meter: A Device for estimating the Subtalar and Talocrural axis of the Human Ankle joint by Applying the Product of Exponentials Formula.

## Óscar Agudelo, Ángel Valera, and Julio Vargas-Riaño



In the following cell the initial values from anthropometric values and statistics are loaded, three points represent a plane parallel to the foot, for the position estimation. The figure shows the normal planes to the ankle joint axes and trajectories mapped, representing circular shapes.

In [ ]:
#Ankle joint forward kinematics by using Product of Exponentials Formula

from sage.plot.plot3d.shapes2 import *
from sage.plot.plot3d.tachyon import ParametricPlot

# Rotation axis in a unitary vector and angle in degrees
def rot_ax_ang(u,a):
    #degrees to radians
    alpha=a*pi/180
    na=(1-cos(alpha))
    ca=cos(alpha)
    sa=sin(alpha)
    #rotation matrix
    R=matrix([[u[0]^2*na+ca, u[0]*u[1]*na-u[2]*sa, u[0]*u[2]*na+u[1]*sa],
            [u[0]*u[1]*na+u[2]*sa, u[1]^2*na+ca, u[1]*u[2]*na-u[0]*sa],
            [u[2]*u[0]*na-u[1]*sa, u[2]*u[1]*na+u[0]*sa, u[2]^2*na+ca]])
    return R

pvar=0.9 #percentual variation

#Talocrural direction
TCa1=-6*pvar
TCa2=80*pvar
vTC=rot_ax_ang([0,0,1], TCa1)*rot_ax_ang([1,0,0], TCa2)*vector([0,0,1])
#Subtalar direction
STa1=23*pvar
STa2=41*pvar
vST=rot_ax_ang([0,0,1], STa1)*rot_ax_ang([0,1,0], STa2)*vector([1,0,0])

#Anthropometric values
#vK=12;vL=11;vO=16;vP=1;vQ=5;vR=0.54 #anthropometric measurements
vK=12*pvar;vL=11*pvar;vO=16*pvar;vP=1*pvar;vQ=5*pvar;vR=0.54*pvar #anthropometric measurements
#H=1750 #heigth in mm
H=1750*pvar #heigth in mm
d_m=(0.285-0.039)*H/2 #knee-ankle half distance
d_p=0.039*H #ankle-foot
z_p=-(d_m+d_p) # z initial position 
r_p=0.055*H*2/3 #platform radius 
ae=2*pi/3 #auxiliar angle
PO=vector([0,0,0]) #Origin at middle shank

#initial platform position
A0=vector([r_p,0,z_p]) 
B0=vector([r_p*cos(ae),r_p*sin(ae),z_p])
C0=vector([r_p*cos(-ae),r_p*sin(-ae),z_p])
PM0=(A0+B0+C0)/3 #center of the platform

#marker representation
dot=point3d((0,0,0),size=5,color='red',opacity=.5)
dPO=dot.translate(PO)
dA0=dot.translate(A0)
dB0=dot.translate(B0)
dC0=dot.translate(C0)
dPM0=dot.translate(PM0)

r_1=vector([0,0,-d_m]) #intersection point between the talocrural axis and the sagittal plane
ap=0.039*H 
lp=0.152*H

#points on the malleolar medial and lateral
M_1=r_1-ap*vR*vTC
M_2=r_1+ap*(1-vR)*vTC
tht=(STa2)

#subtalar axis and the sagittal plane intersection 
r_2=vQ*vector([-cos(tht),0,-sin(tht)])+r_1

#points from the hindfoot to the midfoot
N_1=r_2+0.6*lp*vR*vST
N_2=r_2-0.3*lp*vR*vST

#representation of the kinematic chain
lrt=line([PO,r_1])
lm12=line([M_1,M_2])
ln12=line([N_1,N_2])
lrA=line([r_2,A0])
lrB=line([r_2,B0])
lrC=line([r_2,C0])

#markers representation
dr1=dot.translate(r_1)
dr2=dot.translate(r_2)
dM1=dot.translate(M_1)
dM2=dot.translate(M_2)
dN1=dot.translate(N_1)
dN2=dot.translate(N_2)

#reference labels
kwds = {'color':'black','fontsize':24,'fontweight':'bold','rotation':30}
plbl=vector([0,0,8])
lbp = text3d('PO',PO+plbl,**kwds)
lbp += text3d('A0',A0++plbl,**kwds)
lbp += text3d('B0',B0++plbl,**kwds)
lbp += text3d('C0',C0++plbl,**kwds)
lbp += text3d('PM0',PM0++plbl,**kwds)
#axes labels
lbf = text3d('M1',M_1++plbl,**kwds)
lbf += text3d('M2',M_2++plbl,**kwds)
lbf += text3d('N1',N_1++plbl,**kwds)
lbf += text3d('N2',N_2++plbl,**kwds)
lbf += text3d('r1',r_1++plbl,**kwds)
lbf += text3d('r2',r_2++plbl,**kwds)


#finding the twist unitary vectors 
w1=(M_1-M_2)/abs(M_1-M_2)
w2=(N_1-N_2)/abs(N_1-N_2)

#the perpendicular component
v1=-w1.cross_product(r_1)
v2=-w2.cross_product(r_2)

#angles for the talocrural and subtalar rotations
theta1=var('theta_1')
theta2=var('theta_2')

#six dimensional vector xi mapping 
xi1=matrix([v1[0],v1[1],v1[2],w1[0],w1[1],w1[2]]).transpose()
xi2=matrix([v2[0],v2[1],v2[2],w2[0],w2[1],w2[2]]).transpose()

#transformation of exponential matrix of rotation
Rexp1=rot_ax_ang(w1,theta1)

#rotation matrix component of the homogeneous transformation
vexp1=(matrix.identity(3)-Rexp1)*(w1.cross_product(v1))

#conformation of the homogeneous transformation matrix
MTH1=(Rexp1.augment(vexp1)).stack(vector([0,0,0,1]))

#components for the subtalar axis
Rexp2=rot_ax_ang(w2,theta2)
vexp2=(matrix.identity(3)-Rexp2)*(w2.cross_product(v2))
MTH2=(Rexp2.augment(vexp2)).stack(vector([0,0,0,1]))

#transformation matrix representing the initial position
gst0A=matrix([[1,0,0,A0[0]],[0,1,0,A0[1]],[0,0,1,A0[2]],[0,0,0,1]])
gst0B=matrix([[1,0,0,B0[0]],[0,1,0,B0[1]],[0,0,1,B0[2]],[0,0,0,1]])
gst0C=matrix([[1,0,0,C0[0]],[0,1,0,C0[1]],[0,0,1,C0[2]],[0,0,0,1]])
gst0PM=matrix([[1,0,0,PM0[0]],[0,1,0,PM0[1]],[0,0,1,PM0[2]],[0,0,0,1]])

#product of exponential matrices for all the points
MTHA=MTH1*MTH2*gst0A
MTHB=MTH1*MTH2*gst0B
MTHC=MTH1*MTH2*gst0C
MTHPM=MTH1*MTH2*gst0PM

#components of the group of rigid movements for the central point
f_xpm=MTHPM[0][3]
f_ypm=MTHPM[1][3]
f_zpm=MTHPM[2][3]
#orthogonal direction vectors
f_spm=vector([MTHPM[0][0],MTHPM[1][0],MTHPM[2][0]])
f_npm=vector([MTHPM[0][1],MTHPM[1][1],MTHPM[2][1]])
f_apm=vector([MTHPM[0][2],MTHPM[1][2],MTHPM[2][2]])

#components of the three vertices of the plattform
f_xA=MTHA[0][3]
f_yA=MTHA[1][3]
f_zA=MTHA[2][3]
f_xB=MTHB[0][3]
f_yB=MTHB[1][3]
f_zB=MTHB[2][3]
f_xC=MTHC[0][3]
f_yC=MTHC[1][3]
f_zC=MTHC[2][3]

sc=40 #scale factor
ar1=arrow3d(PO,r_1,sc,color='cyan')
ar2=arrow3d(PO,r_2,sc,color='magenta')
aw1=arrow3d(r_1, r_1 + sc*w1,sc, color ='black')
aw2=arrow3d(r_2, r_2 + sc*w2,sc, color = 'black')
av1=arrow3d(r_1, r_1 + sc*v1/abs(r_1),sc, color ='black')
av2=arrow3d(r_2, r_2 + sc*v2/abs(r_2),sc, color ='black')
#orientation frame at PM0    
spm=vector([f_spm[0].subs(theta1==0,theta2==0),
            f_spm[1].subs(theta1==0,theta2==0),
            f_spm[2].subs(theta1==0,theta2==0)]).n()
npm=vector([f_npm[0].subs(theta1==0,theta2==0),
           f_npm[1].subs(theta1==0,theta2==0),
           f_npm[2].subs(theta1==0,theta2==0)]).n()
apm=vector([f_apm[0].subs(theta1==0,theta2==0),
           f_apm[1].subs(theta1==0,theta2==0),
           f_apm[2].subs(theta1==0,theta2==0)])
spmv=arrow3d(PM0, PM0 + sc*spm,sc, color ='red')
npmv=arrow3d(PM0, PM0 + sc*npm,sc, color ='green')
apmv=arrow3d(PM0, PM0 + sc*apm,sc, color ='blue')

scrws = ar1 +ar2+ aw1+aw2 + av1 + av2 + spmv + npmv + apmv

#noise matrix
rdm = random_matrix(ZZ, 3, 3)
rdm = (rdm.augment(vector([0,0,0]))).stack(vector([0,0,0,0]))

#trajectory holding constant angles
dp=30
t2kpm=MTHPM.subs(theta2==0)
trkt2=parametric_plot3d([t2kpm[0][3],t2kpm[1][3],t2kpm[2][3]],
                        (theta_1,-15,15),
                        texture= 'red', thickness='10', plot_points=dp,
                        radius=1)
t1kpm=MTHPM.subs(theta1==0)
trkt1=parametric_plot3d([t1kpm[0][3],t1kpm[1][3],t1kpm[2][3]],
                        (theta_2,-15,15),
                        texture= 'green', thickness='10', plot_points=dp,
                        radius=1)

#normal plane to TC axis
tcpl = implicit_plot3d(lambda x,y,z: w1.dot_product(vector([x,y,z])-PM0), 
                       (-30+PM0[0],30+PM0[0]), (-30+PM0[1],30+PM0[1]), 
                       (-30+PM0[2],30+PM0[2]),color='green',opacity=0.5)
#normal plane to ST axis
stpl = implicit_plot3d(lambda x,y,z: w2.dot_product(vector([x,y,z])-PM0), 
                       (-30+PM0[0],30+PM0[0]), (-30+PM0[1],30+PM0[1]), 
                       (-30+PM0[2],30+PM0[2]), color ='red',opacity=0.5)

#trajectory angles depending on parameter t
dp=30
tmax=15
var('t')
tpm=MTHPM.subs(theta1==t,theta2==t)
vectpm = vector([tpm[0][3],tpm[1][3],tpm[2][3]])
ttpm=parametric_plot3d(vectpm, (t,0,tmax),
                        texture= 'blue', thickness='10', plot_points=dp,
                      radius=1)
#mapped trajectory
prjtr=vectpm - vectpm.dot_product(apm)*(apm)
pltrj=parametric_plot3d(prjtr  + PM0, (t,0,tmax),
                        texture= 'cyan', thickness='10', plot_points=dp,
                       radius=1)
#mapped trajectory to w1
prjw1=prjtr - prjtr.dot_product(w1)*(w1)
pltw1=parametric_plot3d(prjw1  + PM0, (t,0,tmax),
                        texture= 'yellow', thickness='10', plot_points=dp,
                       radius=1)

#mapped trajectory to w2
prjw2=prjtr - prjtr.dot_product(w2)*(w2)
pltw2=parametric_plot3d(prjw2  + PM0, (t,0,tmax),
                        texture= 'yellow', thickness='10', plot_points=dp,
                       radius=1)
#platform initial position
pltf= polygon([A0,B0,C0],color='gray',opacity=1)

#show the initial position, planes and trajectory projections
gr = dPO+dA0+dB0+dC0+dPM0 + tcpl + stpl + ttpm #+ pltrj + pltw1 + pltw2
gr+= lrt+lm12+ln12+lrA+lrB+lrC
gr+= dM1+dM2+dr1+dr2+dN1+dN2


gr+= lbp+lbf+scrws+pltf
gr.show(viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])

The following cell plot a manifold representation of the platform central point. Also a trajectory mapped to the surface in a constant direction.

In [ ]:
#manifold PM 
dp=100
tmin=0
tmax=360
steps=tmax/20
t=var('t')

tpm=MTHPM.subs(theta1==t,theta2==t)
vectpm = vector([tpm[0][3],tpm[1][3],tpm[2][3]])
ttpm=parametric_plot3d(vectpm, (t,0,tmax),
                       texture= 'blue', 
                       thickness='20', 
                       radius = 3,
                       plot_points=dp)
for t1 in srange(tmin,tmax,steps):
    tpm=MTHPM.subs(theta1==t1,theta2==t)
    vectpm = vector([tpm[0][3],tpm[1][3],tpm[2][3]])
    ttpm += parametric_plot3d(vectpm, (t,tmin,tmax),
                              texture= 'magenta', 
                              thickness='10',
                              radius = 1,
                              plot_points=dp)
for t1 in srange(tmin,tmax,steps):
    tpm=MTHPM.subs(theta1==t,theta2==t1)
    vectpm = vector([tpm[0][3],tpm[1][3],tpm[2][3]])
    ttpm += parametric_plot3d(vectpm, (t,tmin,tmax),
                              texture= 'cyan', 
                               thickness='10',
                              radius = 1,
                              plot_points=dp)

kwds = {'color':'black','fontsize':24,'fontweight':'bold','rotation':30}
lbpo = text3d('A0',A0++plbl,**kwds)
lbpo += text3d('B0',B0++plbl,**kwds)
lbpo += text3d('C0',C0++plbl,**kwds)
lbpo += text3d('PM0',PM0++plbl,**kwds)
ttpm+=aw1+aw2 + av1 + av2 + spmv + npmv + apmv + pltf
ttpm.save('pmsim2.png',viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])
ttpm.show(viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])

The following cell plots the manifold representation for the attachment point A.

In [ ]:
#manifold A
dp=100
t=var('t')

ta=MTHA.subs(theta1==t,theta2==t)
vecta = vector([ta[0][3],ta[1][3],ta[2][3]])
tta=parametric_plot3d(vecta, (t,0,tmax),
                        texture= 'blue', thickness='20', plot_points=dp,
                     radius = 3)
for t1 in srange(tmin,tmax,steps):
    ta=MTHA.subs(theta1==t1,theta2==t)
    vecta = vector([ta[0][3],ta[1][3],ta[2][3]])
    tta += parametric_plot3d(vecta, (t,tmin,tmax),
                        texture= 'magenta', thickness='10', plot_points=dp,
                            radius = 1)
for t1 in srange(tmin,tmax,steps):
    ta=MTHA.subs(theta1==t,theta2==t1)
    vecta = vector([ta[0][3],ta[1][3],ta[2][3]])
    tta += parametric_plot3d(vecta, (t,tmin,tmax),
                        texture= 'cyan', thickness='10', plot_points=dp,
                           radius = 1,)
#components of the group of rigid movements for the central point
f_xa=MTHA[0][3]
f_ya=MTHA[1][3]
f_za=MTHA[2][3]
#orthogonal direction vectors
f_sa=vector([MTHA[0][0],MTHA[1][0],MTHA[2][0]])
f_na=vector([MTHA[0][1],MTHA[1][1],MTHA[2][1]])
f_aa=vector([MTHA[0][2],MTHA[1][2],MTHA[2][2]])
#orientation frame at PM0    
sa=vector([f_sa[0].subs(theta1==0,theta2==0),
            f_sa[1].subs(theta1==0,theta2==0),
            f_sa[2].subs(theta1==0,theta2==0)]).n()
na=vector([f_na[0].subs(theta1==0,theta2==0),
           f_na[1].subs(theta1==0,theta2==0),
           f_na[2].subs(theta1==0,theta2==0)]).n()
aa=vector([f_aa[0].subs(theta1==0,theta2==0),
           f_aa[1].subs(theta1==0,theta2==0),
           f_aa[2].subs(theta1==0,theta2==0)])
sav=arrow3d(A0, A0 + sc*sa,sc, color ='red')
nav=arrow3d(A0, A0 + sc*na,sc, color ='green')
aav=arrow3d(A0, A0 + sc*aa,sc, color ='blue')

tta+=aw1+aw2 + av1 + av2 + sav + nav + aav + pltf
tta.save('asim2.png',viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])
tta.show(viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])

The following cell shows the manifold for the attaching point B

In [ ]:
#manifold B
dp=100
tmin=0
tmax=360
t=var('t')

tb=MTHB.subs(theta1==t,theta2==t)
vectb = vector([tb[0][3],tb[1][3],tb[2][3]])
ttb=parametric_plot3d(vectb, (t,0,tmax),
                        texture= 'blue', thickness='40', plot_points=dp,
                    radius = 3)
for t1 in srange(tmin,tmax,steps):
    tb=MTHB.subs(theta1==t1,theta2==t)
    vectb = vector([tb[0][3],tb[1][3],tb[2][3]])
    ttb += parametric_plot3d(vectb, (t,tmin,tmax),
                        texture= 'magenta', thickness='20', plot_points=dp,
                            radius = 1)
for t1 in srange(tmin,tmax,steps):
    tb=MTHB.subs(theta1==t,theta2==t1)
    vectb = vector([tb[0][3],tb[1][3],tb[2][3]])
    ttb += parametric_plot3d(vectb, (t,tmin,tmax),
                        texture= 'cyan', thickness='20', plot_points=dp,
                           radius = 1)

#components of the group of rigid movements 
f_xb=MTHB[0][3]
f_yb=MTHB[1][3]
f_zb=MTHB[2][3]
#orthogonal direction vectors
f_sb=vector([MTHB[0][0],MTHB[1][0],MTHB[2][0]])
f_nb=vector([MTHB[0][1],MTHB[1][1],MTHB[2][1]])
f_ab=vector([MTHB[0][2],MTHB[1][2],MTHB[2][2]])
#orientation frame at B0    
sb=vector([f_sb[0].subs(theta1==0,theta2==0),
            f_sb[1].subs(theta1==0,theta2==0),
            f_sb[2].subs(theta1==0,theta2==0)]).n()
nb=vector([f_nb[0].subs(theta1==0,theta2==0),
           f_nb[1].subs(theta1==0,theta2==0),
           f_nb[2].subs(theta1==0,theta2==0)]).n()
ab=vector([f_ab[0].subs(theta1==0,theta2==0),
           f_ab[1].subs(theta1==0,theta2==0),
           f_ab[2].subs(theta1==0,theta2==0)])
sbv=arrow3d(B0, B0 + sc*sa,sc, color ='red')
nbv=arrow3d(B0, B0 + sc*na,sc, color ='green')
abv=arrow3d(B0, B0 + sc*aa,sc, color ='blue')

ttb+=aw1+aw2 + av1 + av2 + sbv + nbv + abv + pltf

ttb.save('bsim2.png',viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])
ttb.show(viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])

The following cell plots the manifold for the point C

In [ ]:
#manifold c
dp=100
tmin=0
tmax=360
t=var('t')

tc=MTHC.subs(theta1==t,theta2==t)
vectc = vector([tc[0][3],tc[1][3],tc[2][3]])
ttc=parametric_plot3d(vectc, (t,0,tmax),
                        texture= 'blue', thickness='20', plot_points=dp,
                        radius = 3)
for t1 in srange(tmin,tmax,steps):
    tc=MTHC.subs(theta1==t1,theta2==t)
    vectc = vector([tc[0][3],tc[1][3],tc[2][3]])
    ttc+= parametric_plot3d(vectc, (t,tmin,tmax),
                        texture= 'magenta', thickness='10', plot_points=dp,
                        radius = 1)
for t1 in srange(tmin,tmax,steps):
    tc=MTHC.subs(theta1==t,theta2==t1)
    vectc = vector([tc[0][3],tc[1][3],tc[2][3]])
    ttc += parametric_plot3d(vectc, (t,tmin,tmax),
                        texture= 'cyan', thickness='10', plot_points=dp,
                        radius = 1)
    
#components of the group of rigid movements 
f_xc=MTHC[0][3]
f_yc=MTHC[1][3]
f_zc=MTHC[2][3]
#orthogonal direction vectors
f_sc=vector([MTHC[0][0],MTHC[1][0],MTHC[2][0]])
f_nc=vector([MTHC[0][1],MTHC[1][1],MTHC[2][1]])
f_ac=vector([MTHC[0][2],MTHC[1][2],MTHC[2][2]])
#orientation frame at C0    
scp=vector([f_sc[0].subs(theta1==0,theta2==0),
            f_sc[1].subs(theta1==0,theta2==0),
            f_sc[2].subs(theta1==0,theta2==0)]).n()
nc=vector([f_nc[0].subs(theta1==0,theta2==0),
           f_nc[1].subs(theta1==0,theta2==0),
           f_nc[2].subs(theta1==0,theta2==0)]).n()
ac=vector([f_ac[0].subs(theta1==0,theta2==0),
           f_ac[1].subs(theta1==0,theta2==0),
           f_ac[2].subs(theta1==0,theta2==0)])
scv=arrow3d(C0, C0 + sc*scp,sc, color ='red')
ncv=arrow3d(C0, C0 + sc*nc,sc, color ='green')
acv=arrow3d(C0, C0 + sc*ac,sc, color ='blue')

ttc+=aw1+aw2 + av1 + av2 + scv + ncv + acv + pltf

ttc.save('csim2.png',viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])
ttc.show(viewer='tachyon', antialiasing=True, raydepth=3,
    figsize=[60,60], # the image resolution is 100*figsize
    camera_position=[3, -4, 2], # a distant camera position combined with 
    zoom=2, # a large zoom factor will decrease perspective distortion.
    updir=(0, 0, 2), # the camera is slightly tilted
    viewdir=(-2.2,3.,-1.5), # slightly off-center
    light_position=(4.0, -4.0, 3.0),
    frame=false,axes = false,aspect_ratio=[1,1,1])

This cell export the symbolics expresions for computing the mainfolds in codes like asympote or C

In [ ]:
#saving symbolic expressions to Asymptote file output 
tx, ty =var('t_x' ,'t_y')
pmx=f_xpm.subs(theta1==tx, theta2==ty)
pmy=f_ypm.subs(theta1==tx, theta2==ty)
pmz=f_zpm.subs(theta1==tx, theta2==ty)
ax=f_xA.subs(theta1==tx, theta2==ty)
ay=f_yA.subs(theta1==tx, theta2==ty)
az=f_zA.subs(theta1==tx, theta2==ty)
bx=f_xB.subs(theta1==tx, theta2==ty)
by=f_yB.subs(theta1==tx, theta2==ty)
bz=f_zB.subs(theta1==tx, theta2==ty)
cx=f_xC.subs(theta1==tx, theta2==ty)
cy=f_yC.subs(theta1==tx, theta2==ty)
cz=f_zC.subs(theta1==tx, theta2==ty)
#exportg = [[pmx,pmy,pmz],[ax,ay,az],[bx,by,bz],[cx,cy,cz], 
#           [A0.n(),B0.n(),C0.n(),PM0.n()], 
#           [r_1.n(),r_2.n(),N_1.n(),N_2.n(),M_1.n(),M_2.n()]]
#with  open("outasymptote.txt", "w") as f: f.write(str(exportg))

The following cell shows interactivelly the angular limits

In [ ]:
#limit values
t1min=-15
t1max=15
t2min=-15
t2max=15
#changing the angular parameters interactively 
@interact
def _(t1mi=slider(-90,-1,label=r'min $\theta_1$', default=-15), 
      t1ma=slider(1,90,label=r'max $\theta_1$', default=15),
      t2mi=slider(-90,-1,label=r'min $\theta_2$', default=-15), 
      t2ma=slider(1,90,label=r'max $\theta_2$', default=15)):
    global t1min,t1max,t2min,t2max
     #substitute values
    t1min=t1mi
    t1max=t1ma
    t2min=t2mi
    t2max=t2ma

The following cell plots the surfaces for different angles

In [ ]:
#change angle values
@interact
def _(txi=slider(t1min,t1max, step_size=1, label=r'Value $\theta_1$', default=0), 
    tyi=slider(t2min,t2max, step_size=1, label=r'Value $\theta_2$', default=0)):
    pmxs=pmx.subs(tx==txi, ty==tyi)
    pmys=pmy.subs(tx==txi, ty==tyi)
    pmzs=pmz.subs(tx==txi, ty==tyi)
    axs=ax.subs(tx==txi, ty==tyi)
    ays=ay.subs(tx==txi, ty==tyi)
    azs=az.subs(tx==txi, ty==tyi)
    bxs=bx.subs(tx==txi, ty==tyi)
    bys=by.subs(tx==txi, ty==tyi)
    bzs=bz.subs(tx==txi, ty==tyi)
    cxs=cx.subs(tx==txi, ty==tyi)
    cys=cy.subs(tx==txi, ty==tyi)
    czs=cz.subs(tx==txi, ty==tyi)
    #plotting
    sc=50
    dp=30 #data points
    cpm=parametric_plot3d([f_xpm, f_ypm, f_zpm],(theta_1, t1min, t1max), 
                        (theta_2, t2min, t2max),plot_points=[dp,dp])
    cA=parametric_plot3d([f_xA, f_yA, f_zA], (theta_1, t1min, t1max), 
                       (theta_2, t2min, t2max),plot_points=[dp,dp])
    cB=parametric_plot3d([f_xB, f_yB, f_zB], (theta_1, t1min, t1max), 
                       (theta_2, t2min, t2max),plot_points=[dp,dp])
    cC=parametric_plot3d([f_xC, f_yC, f_zC], (theta_1, t1min, t1max), 
                       (theta_2, t2min, t2max),plot_points=[dp,dp],texture="red")

    ptpm=vector([pmxs,pmys,pmzs])
    spms=vector([f_spm[0].subs(theta1==txi, theta2==tyi),
                 f_spm[1].subs(theta1==txi, theta2==tyi),
                 f_spm[2].subs(theta1==txi, theta2==tyi)]).n()
    apms=arrow3d(ptpm, ptpm + sc*spms,sc, color ='red')
    spmn=vector([f_npm[0].subs(theta1==txi, theta2==tyi),
                 f_npm[1].subs(theta1==txi, theta2==tyi),
                 f_npm[2].subs(theta1==txi, theta2==tyi)]).n()
    apmn=arrow3d(ptpm, ptpm + sc*spmn,sc, color ='green')  
    spma=vector([f_apm[0].subs(theta1==txi, theta2==tyi), 
                 f_apm[1].subs(theta1==txi, theta2==tyi),
                 f_apm[2].subs(theta1==txi, theta2==tyi)]).n()
    apma=arrow3d(ptpm, ptpm + sc*spma,sc, color ='blue')   
    pta=vector([axs,ays,azs]).n()
    ptb=vector([bxs,bys,bzs]).n()
    ptc=vector([cxs,cys,czs]).n()
    ptf=polygon([pta,ptb,ptc],color='green',opacity=0.7) 
        #plotting
    fkin=apma+apmn+apms+ptf+cA+cB+cC+cpm
    fkin.show(viewer='tachyon', antialiasing=True, raydepth=3,
        figsize=[90,50], # the image resolution is 100*figsize
        camera_position=[3, 4, 2], # a distant camera position combined with 
        zoom=4.5, # a large zoom factor will decrease perspective distortion.
        updir=(0, 0, 1), # the camera is slightly tilted
        viewdir=(-2.4,-3.,-1.6), # slightly off-center
        light_position=(4.0, -4.0, 3.0),
        frame=false,axes = false,aspect_ratio=1
        )
    fkin.save('interactive1.png', viewer='tachyon', antialiasing=True, raydepth=3,
        figsize=[90,50], # the image resolution is 100*figsize
        camera_position=[3, 4, 2], # a distant camera position combined with 
        zoom=4.5, # a large zoom factor will decrease perspective distortion.
        updir=(0, 0, 1), # the camera is slightly tilted
        viewdir=(-2.4,-3.,-1.6), # slightly off-center
        light_position=(4.0, -4.0, 3.0),
        frame=false,axes = false,aspect_ratio=1
        )
    print("pm in w1 comp:", [ptpm.dot_product(w1).n(),spmn.dot_product(w1).n(),spma.dot_product(w1).n()])
    print("pm in w2 comp:", [ptpm.dot_product(w2).n(),spmn.dot_product(w2).n(),spma.dot_product(w2).n()])

The following cell presents numerically a trajectory

In [ ]:
#Trajectory t same rate
t=var('t')

tpm=MTHPM.subs(theta1==t,theta2==t, pi==3.14)

f = lambda t: (-24.67*cos(t)^2 - 57.38*cos(t)*sin(t) +
               11.97*sin(t)^2 + 22.86*cos(t) - 17.35*sin(t) + 1.81,
               -9.55*cos(t)^2 + 16.33*cos(t)*sin(t) + 
               5.22*sin(t)^2 - 9.61*cos(t) + 30.93*sin(t) + 19.17,
               -36.33*cos(t)^2 + 33.37*cos(t)*sin(t) + 
               19.44*sin(t)^2 - 18.81*cos(t) - 34.39*sin(t) - 199.99)
               

t = Tachyon(xres=900, yres=600, zoom=1.0, antialiasing=10, 
            aspectratio=1.0, raydepth=8, camera_position=[100,100,-30], 
            camera_center=None, updir=[0, 0, 10], look_at=[0, 0, 0], 
            viewdir=[-0.4,-0.4,-0.6], projection='PERSPECTIVE', focallength='', 
            aperture='', frustum=''
           )

t.light((10,10,10), 0.8, (1,1,1))

t.texture('t',  ambient=0.5, diffuse=0.9, specular=0.5, opacity=1.0, color=(1,0,0))
t.texture('t0',  ambient=0.5, diffuse=0.9, specular=0.5, opacity=1.0, color=(0,1,0))
t.parametric_plot(f, 0, 2*pi, 't', r=1, cylinders=True, min_depth=5, max_depth=8, e_rel=0.01, e_abs=0.01)

t.show()